In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [5]:
import matplotlib as plt
import tensorflow as tf
import seaborn as sb
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Activation
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import optimizers, initializers, layers
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter(action="ignore")

In [6]:
train = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
test = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")

In [7]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [9]:
train.shape

(159571, 8)

In [10]:
test.shape

(153164, 2)

In [11]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
test.describe()

,id,comment_text
count,153164,153164
unique,153164,153164
top,cab7d96a4020887a,: How kind of you. It's okay.
freq,1,1


In [13]:
train.info

<bound method DataFrame.info of                       id                                       comment_text  \
0       0000997932d777bf  Explanation\nWhy the edits made under my usern...   
1       000103f0d9cfb60f  D'aww! He matches this background colour I'm s...   
2       000113f07ec002fd  Hey man, I'm really not trying to edit war. It...   
3       0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...   
4       0001d958c54c6e35  You, sir, are my hero. Any chance you remember...   
...                  ...                                                ...   
159566  ffe987279560d7ff  ":::::And for the second time of asking, when ...   
159567  ffea4adeee384e90  You should be ashamed of yourself \n\nThat is ...   
159568  ffee36eab5c267c9  Spitzer \n\nUmm, theres no actual article for ...   
159569  fff125370e4aaaf3  And it looks like it was actually you who put ...   
159570  fff46fc426af1f9a  "\nAnd ... I really don't think you understand...   

        toxic  seve

In [14]:
train.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import TreebankWordTokenizer

In [17]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    tokenizer=TreebankWordTokenizer()
    comment_tokens=tokenizer.tokenize(text)
    return comment_tokens

In [18]:
train['comment_text'] = train['comment_text'].apply(lambda x: clean_text(x))
test['comment_text']= test['comment_text'].apply(lambda x: clean_text(x))
train['comment_text'].head()
test['comment_text'].head()

0    [yo, bitch, ja, rule, is, more, succesful, the...
1    [from, rfc, the, title, is, fine, as, it, is, ...
2              [sources, zawe, ashton, on, lapland, —]
3    [if, you, have, a, look, back, at, the, source...
4      [i, dont, anonymously, edit, articles, at, all]
Name: comment_text, dtype: object

In [19]:
def remove_stopwords(text):
    stemmer=PorterStemmer()
    words=[]
    for word in text:
        if(word not in stopwords.words('english')):
            stem_word=stemmer.stem(word)
            words.append(stem_word)
    return words

In [25]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(10000000)

10000000


In [26]:
train['comment_text'] = train['comment_text'].apply(lambda x: remove_stopwords(x))
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"[explanationwhi, edit, made, usernam, hardcor,...",0,0,0,0,0,0
1,000103f0d9cfb60f,"[daww, match, background, colour, im, seemingl...",0,0,0,0,0,0
2,000113f07ec002fd,"[hey, man, im, realli, tri, edit, war, guy, co...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"[morei, cant, make, real, suggest, improv, won...",0,0,0,0,0,0
4,0001d958c54c6e35,"[sir, hero, chanc, rememb, page, that]",0,0,0,0,0,0


In [27]:
train.to_csv('train.csv')

In [28]:
test['comment_text']=test['comment_text'].apply(lambda x: remove_stopwords(x))
test.head()

,id,comment_text
0,00001cee341fdb12,"[yo, bitch, ja, rule, succes, youll, ever, wha..."
1,0000247867823ef7,"[rfc, titl, fine, imo]"
2,00013b17ad220c46,"[sourc, zaw, ashton, lapland, —]"
3,00017563c3f7919a,"[look, back, sourc, inform, updat, correct, fo..."
4,00017695ad8997eb,"[dont, anonym, edit, articl]"


In [29]:
test.to_csv('test.csv')

In [41]:
train_df=train['comment_text']
test_df=test['comment_text']

In [42]:
max_features=22000
tokenizer=Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_df))
tokenized_train=tokenizer.texts_to_sequences(train_df)
tokenized_testt=tokenizer.texts_to_sequences(test_df)

In [43]:
maxlen=200
x_train=pad_sequences(tokenized_train,maxlen=maxlen)
x_test=pad_sequences(tokenized_testt, maxlen=maxlen)

In [44]:
embed_size=128
maxlen=200
max_features=22000
inp=Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size)(inp)
x = LSTM(60, return_sequences=True, name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)

In [45]:
model=Model(inputs=inp,outputs=x)
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [46]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 128)          2816000   
_________________________________________________________________
lstm_layer (LSTM)            (None, 200, 60)           45360     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 60)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3050      
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0   

In [47]:
cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
targets = train[cols].values

In [48]:
batch_size=64
epochs=2
model.fit(x_train,targets,batch_size=batch_size,epochs=epochs,validation_split=0.1)

Epoch 1/2
2244/2244 [==============================] - 90s 40ms/step - loss: 0.1389 - accuracy: 0.8006 - val_loss: 0.0530 - val_accuracy: 0.9939
Epoch 2/2
2244/2244 [==============================] - 88s 39ms/step - loss: 0.0488 - accuracy: 0.9873 - val_loss: 0.0512 - val_accuracy: 0.9932


In [49]:
prediction=model.predict(x_test)
prediction

array([[9.95479345e-01, 2.85529912e-01, 9.62349951e-01, 3.22645158e-02,
        8.51169646e-01, 1.20877884e-01],
       [1.57370546e-03, 1.80410382e-06, 3.84288287e-04, 1.04383207e-05,
        2.17050299e-04, 3.58847356e-05],
       [3.85507429e-03, 7.96299901e-06, 9.30969079e-04, 3.19272331e-05,
        6.42704777e-04, 9.84745784e-05],
       ...,
       [1.29722466e-04, 2.46689797e-08, 3.01454129e-05, 2.96047034e-07,
        1.18556591e-05, 1.37657503e-06],
       [4.93375817e-03, 3.11516010e-06, 7.75313820e-04, 2.54111637e-05,
        6.00401603e-04, 1.16901065e-04],
       [9.60548997e-01, 1.92606058e-02, 7.99666584e-01, 2.99055316e-03,
        4.43307042e-01, 1.67279579e-02]], dtype=float32)

In [ ]:
model.save('model')